In [2]:
import json
import pyspark
from pyspark.sql import SparkSession

cfg = json.load(open('cred.json'))
DBUSER = cfg.get('Connect DB')['DBUSER']
DBPASS = cfg.get('Connect DB')['DBPASS']
DBHOST = cfg.get('Connect DB')['DBHOST']
DBPORT = cfg.get('Connect DB')['DBPORT']
DBNAME = cfg.get('Connect DB')['DBNAME']

jar_path = "hdfs://jars/postgresql-42.7.4.jar"

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkByExamples.com") \
    .config("spark.jars", jar_path) \
    .config("spark.driver.extraClassPath", jar_path) \
    .getOrCreate()

In [3]:
def write_by_jdbc(sdf, db_table_name,  cfg):
    DBUSER = cfg.get('Connect DB')['DBUSER']
    DBPASS = cfg.get('Connect DB')['DBPASS']
    DBHOST = cfg.get('Connect DB')['DBHOST']
    DBPORT = cfg.get('Connect DB')['DBPORT']
    DBNAME = cfg.get('Connect DB')['DBNAME']
    
    sdf.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://{DBHOST}:{DBPORT}/{DBNAME}".format(DBHOST = DBHOST, DBPORT=DBPORT, DBNAME=DBNAME)) \
    .option("dbtable", db_table_name) \
    .option("user", DBUSER) \
    .option("password", DBPASS) \
    .option("driver", 'org.postgresql.Driver')\
    .save()

In [4]:
db_table_name = "public.tmp_random_data"
N = 100
# create dataframe random
df = spark.range(N)
sdf = df.selectExpr("id as id")
write_by_jdbc(sdf, db_table_name, cfg)

In [5]:
query_stmt = f"select * from {db_table_name}"
# create dataframe
df = spark.read.format("jdbc") \
    .option("url", f"jdbc:postgresql://{DBHOST}:{DBPORT}/{DBNAME}") \
    .option("query", query_stmt) \
    .option("user", DBUSER) \
    .option("password", DBPASS) \
    .load()
 
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows

